# Imports

In [1]:
#for importing code
import sys, os
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    print(f"Adding {module_path} to sys.path")
    sys.path.append(module_path)

Adding /Users/yonatanlou/dev/meaningful-transparency/src to sys.path


In [2]:
import pandas as pd
from constants import PROJECT_ROOT

twitter_df = pd.read_csv(f"{PROJECT_ROOT}/datasets/GoldStandard2024.csv")
# URL = "https://zenodo.org/records/14448399/files/GoldStandard2024.csv?download=1"  
# twitter_df = pd.read_csv(URL)
twitter_df["CreateDate"] = pd.to_datetime(twitter_df["CreateDate"], errors="coerce")
twitter_df["Year"] = twitter_df["CreateDate"].dt.year
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11311 entries, 0 to 11310
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   ID          11311 non-null  int64              
 1   Username    11311 non-null  object             
 2   CreateDate  11311 non-null  datetime64[ns, UTC]
 3   Biased      11311 non-null  int64              
 4   Keyword     11311 non-null  object             
 5   Text        11311 non-null  object             
 6   Year        11311 non-null  int32              
dtypes: datetime64[ns, UTC](1), int32(1), int64(2), object(3)
memory usage: 574.5+ KB


In [3]:
import re
import numpy as np
import pandas as pd

TEXT_COL = "Text"

URL_RE = re.compile(r"(https?://\S+?)(?=\s|$)", flags=re.IGNORECASE)
URL_EXTERNAL = re.compile(r"(https://t.co/.*\s via)", flags=re.IGNORECASE)
MENTION_RE = re.compile(r"(^|\s)@\w+")
HASHTAG_RE = re.compile(r"(^|\s)#\w+")
SPACE_RE = re.compile(r"\s+")
VIA_RE = re.compile(r"\bvia\b", flags=re.IGNORECASE)

def strip_urls(s: str) -> str:
    return URL_RE.sub(" ", s)

def strip_mentions(s: str) -> str:
    return MENTION_RE.sub(" ", s)

def number_of_mentions(s: str) -> int:
    return len(MENTION_RE.findall(s))

def strip_hashtags(s: str) -> str:
    return HASHTAG_RE.sub(" ", s)

def normalize_space(s: str) -> str:
    return SPACE_RE.sub(" ", s).strip()

def core_text(s: str) -> str:
    s = strip_urls(s)
    s = strip_mentions(s)
    s = strip_hashtags(s)
    return normalize_space(s)

def count_hashtags(s: str) -> int:
    return len(HASHTAG_RE.findall(s))

def count_words(s: str) -> int:
    s = normalize_space(s)
    return 0 if not s else len(s.split())


def url_via_flags(s: str) -> tuple[bool, bool]:
    """Return (non_twitter_url, twitter_url) per your rule:
       non_twitter_url => there is a URL and a 'via' occurs AFTER a URL
       twitter_url     => there is a URL and there is NO 'via' after any URL
    """
    if not s:
        return (False, False)
    has_url = False
    via_after_any_url = False

    for m in URL_RE.finditer(s):
        has_url = True
    
    if URL_EXTERNAL.findall(s):
        via_after_any_url = True

    if not has_url:
        return (False, False)
    if via_after_any_url:
        return (True, False)
    else:
        return (False, True)

def add_filter_flags(df: pd.DataFrame, text_col: str = TEXT_COL) -> pd.DataFrame:
    out = df.copy()
    txt = out[text_col].fillna("").astype(str)

    # Precompute helpers
    txt_no_urls = txt.map(strip_urls).map(normalize_space)
    txt_core     = txt.map(core_text)
    total_tokens = txt.map(count_words)
    core_tokens  = txt_core.map(count_words)
    hashtag_counts = txt.map(count_hashtags)

    # Existing flags
    out["n_words"] = total_tokens
    out["n_words_no_urls_mentions_hashtags"] = core_tokens.astype(int)
    out["extracted_urls"] = txt.map(lambda s: URL_RE.findall(s))
    out["flag_link_only"] = txt_no_urls.str.len().eq(0) | core_tokens.le(2)
    out["flag_starts_with_mention"] = txt.str.match(r"^\s*@\w+")
    out["n_mentions_in_text"] = txt.map(number_of_mentions)
    out["n_hashtags_in_text"] = hashtag_counts
    out["n_urls_in_text"] = txt.map(lambda s: len(URL_RE.findall(s)))
    out["no_text_ratio"] = (out["n_mentions_in_text"]+out["n_hashtags_in_text"]+out["n_urls_in_text"]) / (core_tokens+out["n_mentions_in_text"]+out["n_hashtags_in_text"]+out["n_urls_in_text"])
    out["flag_empty_after_cleanup"] = txt_core.str.len().eq(0)
    

    non_twitter, twitter = zip(*txt.map(url_via_flags))
    out["flag_non_twitter_url"] = np.array(non_twitter, dtype=bool)
    out["flag_twitter_url"] = np.array(twitter, dtype=bool)


    # Optional: expose core text for inspection
    out["Text_core"] = txt_core
    return out




In [4]:
df_flags = add_filter_flags(twitter_df, TEXT_COL)
df_flags


,ID,Username,CreateDate,Biased,Keyword,Text,Year,n_words,n_words_no_urls_mentions_hashtags,extracted_urls,flag_link_only,flag_starts_with_mention,n_mentions_in_text,n_hashtags_in_text,n_urls_in_text,no_text_ratio,flag_empty_after_cleanup,flag_non_twitter_url,flag_twitter_url,Text_core
0,1232258532273090560,amit2nirvana,2020-02-25 10:58:23+00:00,0,Israel,The last 24 hours in Israel: https://t.co/0YNw...,2020,7,6,[https://t.co/0YNwvMsYvL],False,False,0,0,1,0.142857,False,False,True,The last 24 hours in Israel:
1,1224025761092448257,NinjaAlex420,2020-02-02 17:44:17+00:00,0,Israel,@stranahan KnowMoreNews isn't confident enough...,2020,15,13,[https://t.co/rAFfEKOsLo],False,True,1,0,1,0.133333,False,False,True,KnowMoreNews isn't confident enough to debate ...
2,1242382710561107969,SevenShepherd,2020-03-24 09:28:15+00:00,0,Israel,"""[Even] an heritage unto Israel his servant: f...",2020,18,15,[],False,False,0,3,0,0.166667,False,False,False,"""[Even] an heritage unto Israel his servant: f..."
3,1224677205306818561,MGSheikh8,2020-02-04 12:52:53+00:00,0,Israel,Opinion: No One in Israel Knew They Were Commi...,2020,16,16,[https://t.co/2Itgo9cuTE],False,False,0,0,1,0.058824,False,False,True,Opinion: No One in Israel Knew They Were Commi...
4,1214278369338109953,elianebis1,2020-01-06 20:11:38+00:00,0,Israel,https://t.co/LTB0zidzM9 When #TheCloudOfGod ap...,2020,34,32,[https://t.co/LTB0zidzM9],False,False,0,1,1,0.058824,False,False,True,"When appeared in the days of Moses, He took th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11306,1391021336709918722,Abhishe37182949,2021-05-08 13:25:05+00:00,0,Israel,RT @AskAnshul: This is from Israel. Hundreds ...,2021,31,30,[https://t.co/NTYp9uTeNn],False,False,1,0,1,0.062500,False,False,True,RT : This is from Israel. Hundreds of Israelis...
11307,1396970681582198784,elf212,2021-05-24 23:25:39+00:00,0,Israel,RT @SenatorTimScott: Stand with Israel! https:...,2021,6,5,[https://t.co/gNL1ynXVsm],False,False,1,0,1,0.285714,False,False,True,RT : Stand with Israel!
11308,1393554847614468096,GulathiVinod,2021-05-15 13:12:21+00:00,0,Israel,RT @republic: #BREAKING | Hamas building bombe...,2021,38,34,"[https://t.co/jtcy9lPpRd, https://t.co/61SkEDc...",False,False,1,2,2,0.128205,False,False,True,RT : | Hamas building bombed by Israel Forces ...
11309,1388649440014082048,ROQ1LHvMzJBA3GW,2021-05-02 00:20:01+00:00,0,Israel,@ssarawatlismm Love you so much from Israel ðŸ...,2021,8,7,[],False,True,1,0,0,0.125000,False,False,False,Love you so much from Israel ðŸ‡®ðŸ‡±â¤ðŸ‡®ðŸ...


In [21]:
# Tunables
MIN_CLEAN_TEXT_LENGTH = 10
MAX_MENTIONS_IN_TEXT = 1
MAX_NO_TEXT_RATIO = 0.85  # fraction of removed tokens allowed


len_ok = df_flags["n_words_no_urls_mentions_hashtags"] > MIN_CLEAN_TEXT_LENGTH
not_url_only = ~df_flags["flag_link_only"]
not_empty_after = ~df_flags["flag_empty_after_cleanup"]
mentions_ok = df_flags["n_mentions_in_text"] <= MAX_MENTIONS_IN_TEXT
noise_ratio_ok = df_flags["no_text_ratio"] <= MAX_NO_TEXT_RATIO
no_non_twitter_url = ~df_flags["flag_non_twitter_url"]
is_twitter_url = df_flags["flag_twitter_url"]
any_mention_in_text = df_flags["n_mentions_in_text"] > 0

# Combine all rules
row_is_valid = (
    len_ok
    & not_url_only
    & not_empty_after
    & mentions_ok
    & noise_ratio_ok
    & no_non_twitter_url
    & is_twitter_url #~3300 rows will be removed
    # & any_mention_in_text #~5000 rows will be removed
)

# Apply
filtered = df_flags.loc[row_is_valid].copy()

print(f"Filtered from {len(df_flags)} to {len(filtered)} rows ({len(df_flags)-len(filtered)} removed)")
print(filtered["Biased"].value_counts(dropna=False))


Filtered from 11311 to 5048 rows (6263 removed)
Biased
0    4292
1     756
Name: count, dtype: int64




## Rules Applied

1. **Minimum text length**  
   - Text have to be larger than 10 words (not including hashtags, mentions, urls).

2. **Not link-only**  
   - Drop rows with only url.

3. **Not empty after cleanup**  
   - Drop rows with no text after removing url/hashtags/mentions.

4. **Limit on user mentions**  
   - Keep rows with `n_mentions_in_text <= 1`.

5. **Noise ratio cap**  
   - Keep rows where `no_text_ratio <= 0.85`  

6. **Exclude non-Twitter URLs**  
   - Drop rows flagged as `flag_non_twitter_url`  
     (posts that contain external links with “via” attribution).



## Outcome

- **Before filtering:** `11311` rows  
- **After filtering:** `8321` rows  


If we will remove all posts with urls, we will lose more 3300 rows.
If we will remove all posts with any mention of other user, we will lose more 5000 rows.


## Dist by year:

In [22]:
dist_by_year = filtered.groupby("Year")["Biased"].value_counts().reset_index(name="count").pivot(index="Year", columns="Biased", values="count").fillna(0).astype(int)
dist_by_year.columns = ["Not antisemite", "Antisemitic"]
dist_by_year

,Not antisemite,Antisemitic
Year,,
2019,558,168
2020,1470,191
2021,933,123
2022,972,179
2023,359,95
